In [2]:
import cv2
import numpy as np

widthImg, heightImg= 640, 480

In [3]:
def preProcessing(img):
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(img, (5,5), 1)
    imgCanny = cv2.Canny(imgBlur, 150, 120)
    kernel= np.ones((5,5))
    imgDial = cv2.dilate(imgCanny, kernel, iterations = 2)
    imgThres = cv2.erode(imgDial, kernel, iterations = 1)
    return imgThres

In [4]:
def getContours(img):
    maxApprox = np.array([])
    maxArea = 0
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area= cv2.contourArea(cnt)
        if area > 5000:
            peri = cv2.arcLength(cnt, True) # True means it is closed
            approx = cv2.approxPolyDP(cnt, 0.02*peri, True )
            if area > maxArea and len(approx) == 4 :
                maxArea = area
                maxApprox = approx
    cv2.drawContours(imgContour, maxApprox, -1, (0,255,0), 20)  # -1 to show all contours
    return maxApprox            

In [5]:
def reorder(myPoints):
    myPoints = myPoints.reshape((4,2))
    add = myPoints.sum(1)
    diff = np.diff(myPoints, axis = 1)
    
    myNewPoints= np.zeros((4,1,2))
    myNewPoints[0] = myPoints[np.argmin(add)]
    myNewPoints[3] = myPoints[np.argmax(add)]
    myNewPoints[1] = myPoints[np.argmin(diff)]
    myNewPoints[2] = myPoints[np.argmin(diff)]
    return myNewPoints

In [8]:
def getWarp(img, maxApprox):
    maxApprox = reorder (maxApprox)
    src_pts= np.float32(maxApprox)
    des_pts= np.float32(([0,0], [widthImg,0], [0,heightImg], [widthImg,heightImg]))

    matrix = cv2.getPerspectiveTransform(src_pts, des_pts) 
    imgOutput= cv2.warpPerspective(img, matrix, (widthImg,heightImg))
    
    imgCropped = imgOutput[20:imgOutput.shape[0]-20,20:imgOutput.shape[1]-20]
    imgCropped = cv2.resize(imgCropped,(widthImg,heightImg))
    
    return imgCropped

In [11]:
cap = cv2.VideoCapture(0)
cap.set(3, 640)  # set length
cap.set(4, 480)  # set width
cap.set(10,10)  # set brightness


while True:
    succcess, img= cap.read()
    img = cv2.resize(img, (widthImg, heightImg) )
    imgContour = img.copy()
    imgThres = preProcessing(img)
    maxApprox = getContours(imgThres)
    if maxApprox.size != 0:
        imgWarped = getWarp(img, maxApprox)
        cv2.imshow('Doc', imgWarped)
    else:
        cv2.imshow('Webcam', imgThres)

    if cv2.waitKey(1) & 0xff== ord('q'):
        break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()